In [1]:
import thermomatch as match
match.ThermoImpexGenerator.create_local_database_if_not_existent = True

[2025-08-06 17:57:52.383] [jsonio17] [info] Home directory is /home/dmiron


In [2]:
# Set path to `schemas` and `lua` not from the configuration file
match.ThermoImpexGenerator.setResourcesDirectory("../../Resources")

# Connect to Arango local database
generator = match.ThermoImpexGenerator("http://localhost:8529", "root", "", "ORD_TDB2020_v25_test3", True, match.ThermoImpexGenerator.File, True)

[jsonarango] [info] You are connected to: arango 3.9.12


In [3]:
generator.sourcedtbs = (["{\"26\" : \"PSI_NAGRA20\"}"]) 

In [4]:
# List of implemented thermodynamic formats
# print( generator.allFormats() )

# Execute import from PSI table format data
description = [
    match.ImpexRowDescription("icomp", "VertexElement", "records", "FormatTableFile", "scripts-in/elements.FormatTableFile.json"), # element records
    match.ImpexRowDescription("refs", "VertexDataSource", "records", "FormatStructDataFile", "scripts-in/references.FormatStructDataFile.json"), # reference records
    match.ImpexRowDescription("dcomp-aqueous", "VertexSubstance", "records", "FormatTableFile", "scripts-in/master_substances.FormatTableFile.json"), # substance records for master aqeuous species (contain also solids)
   # match.ImpexRowDescription("dcomp-solids", "VertexSubstance", "records", "FormatTableFile", "scripts-in/master_substances.FormatTableFile.json"), # substance records for master solids 
    match.ImpexRowDescription("reacdc-aqueous", "VertexSubstance", "records", "FormatTableFile", "scripts-in/product_substances.FormatTableFile.json"), # substance records for dependent aqeuous species
    match.ImpexRowDescription("reacdc-aqueous", "VertexReaction", "records_and_links", "FormatTableFile", "scripts-in/product_reactions.FormatTableFile.json"), # reaction records for dependent aqeuous species and links to reactants
    match.ImpexRowDescription("reacdc-solids", "VertexSubstance", "records", "FormatTableFile", "scripts-in/product_substances.FormatTableFile.json"), # substance records for dependent solid species 
    match.ImpexRowDescription("reacdc-solids", "VertexReaction", "records_and_links", "FormatTableFile", "scripts-in/product_reactions.FormatTableFile.json"), # reaction records for dependent solid species and links to reactants
    match.ImpexRowDescription("reacdc-solids", "VertexPhase", "records_and_links", "FormatTableFile", "scripts-in/phase-solids.FormatTableFile.json"), # phase records and links to solid substance records
    match.ImpexRowDescription("dcomp-solids", "VertexPhase", "records_and_links", "FormatTableFile", "scripts-in/phase-solids.FormatTableFile.json"), # phase records and links to solid substance records
    #links to references
    match.ImpexRowDescription("dcomp-aqueous", "VertexTemporary", "records_and_links", "FormatTableFile", "scripts-in/master_substances_references.FormatTableFile.json"), # links to master substances from references
    match.ImpexRowDescription("reacdc-aqueous", "VertexTemporary", "records_and_links", "FormatTableFile", "scripts-in/product_reactions_references.FormatTableFile.json"), # links to dependent aqueous substances from references
    match.ImpexRowDescription("reacdc-solids", "VertexTemporary", "records_and_links", "FormatTableFile", "scripts-in/product_reactions_references.FormatTableFile.json"), # links to dependent solid substances from references
    # SIT
    match.ImpexRowDescription("SIT", "VertexPhase", "records", "FormatTableFile", "scripts-in/phase_sit.FormatTableFile.json"), # record for aq_sit phase
    match.ImpexRowDescription("SIT", "VertexMixingModel", "records", "FormatTableFile", "scripts-in/mixmod_sit.FormatTableFile.json"), # record for aq_sit mixing model
    match.ImpexRowDescription("SIT", "VertexInteraction", "records_and_links", "FormatTableFile", "scripts-in/int_Na.FormatTableFile.json"), # interaction records with Na+ and links to substance records
    match.ImpexRowDescription("SIT", "VertexInteraction", "records_and_links", "FormatTableFile", "scripts-in/int_Cl.FormatTableFile.json"), # interaction records with Cl- and links to substance records
    match.ImpexRowDescription("SIT", "VertexInteraction", "records_and_links", "FormatTableFile", "scripts-in/int_ClO4.FormatTableFile.json"), # interaction records with ClO4- and links to substance records
    # references
    # match.ImpexRowDescription("SIT", "VertexTemporary", "records_and_links", "FormatTableFile", "scripts-in/int_Na_references.FormatTableFile.json"), # not working yet
    match.ImpexRowDescription("ThermoDataSet", "VertexThermoDataSet", "records_and_links", "FormatTableFile", "scripts-in/thermodataset.FormatTableFile.json") # ThermoDataSet record and link to elements, substances, rdc
]


In [5]:
#print( description )
#generator.runImport("TableImport", description, "data-in/dcomp-aqueous.aux.csv", "rdc_PSITDB2020");
#generator.runImport("TableImport", description, "data-in/dcomp-aqueous.Pu.csv", "rdc_PSITDB2020");

In [6]:
elements = ['aux','groundwater','aux','Ac','Se','Ag','Am','Cd','Cf','Cm','Cu','Fe','Hg','Mo','Nb',
            'Ni','Zn','U','Np','P','Pa','Pb','Pd','Po','Pu','Ra','SmEuHo','Si','Sn','Tc','Th','Ti','Zr', 'organics', 'Ac', 'Se', 'U', 'Si'] # few elements are repeated due to missing master species in reactions

In [7]:
import concurrent.futures
import sys, time

# Function to process each element (e.g., creating the file name)
def process_element(element):
    file_name = f"data-in/dcomp-aqueous.{element}.csv"
    print(file_name)
    generator.runImport("TableImport", description, file_name, "rdc_PSITDB2020");
    sys.stdout.flush()  # Flush the output buffer
    time.sleep(5)  # Simulate some delay in processing

# Define the number of threads
num_threads = 1  # Set the desired number of threads

# Using ThreadPoolExecutor with a defined number of threads
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Submit each task to the thread pool
    executor.map(process_element, elements)

data-in/dcomp-aqueous.aux.csv
[2025-08-06 17:57:54.548] [thermomatch] [info]  0% Preparing files ...
[2025-08-06 17:57:54.560] [thermomatch] [info]  10% Importing data ...
[2025-08-06 17:57:54.560] [thermomatch] [info]  10% Importing icomp VertexElement ...
[jsonio17] [info] VertexElement loading collection: 35, loading query: 1276
[2025-08-06 17:57:54.684] [thermomatch] [info]  13% Importing refs VertexDataSource ...
[2025-08-06 17:57:54.684] [thermomatch] [info]  16% Importing dcomp-aqueous VertexSubstance ...
[jsonio17] [info] VertexSubstance loading collection: 18, loading query: 1593
[2025-08-06 17:57:54.721] [thermomatch] [info]  19% Importing reacdc-aqueous VertexSubstance ...
[2025-08-06 17:57:54.788] [thermomatch] [info]  22% Importing reacdc-aqueous VertexReaction ...
[jsonio17] [info] VertexReaction loading collection: 12, loading query: 856
[2025-08-06 17:57:54.860] [thermomatch] [info]  25% Importing reacdc-solids VertexSubstance ...
[2025-08-06 17:57:54.875] [thermomatch]

In [8]:
print("Finish")

Finish
